##setup

In [1]:
%%capture
!pip install transformers
!pip install shap

In [15]:
!cp /content/drive/MyDrive/fake-news-adversarial-benchmark/utils_fake_news.py .
%run utils_fake_news.py

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np 
import scipy as sp
import shap
import pickle
import torch
# Device
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

In [3]:
print(torch.cuda.get_device_name(device))

NVIDIA A100-SXM4-40GB


In [4]:
!cp /content/drive/MyDrive/fake-news-adversarial-benchmark/utils_fake_news.py .
%run utils_fake_news.py

cp: cannot stat '/content/drive/MyDrive/fake-news-adversarial-benchmark/utils_fake_news.py': No such file or directory


ERROR:root:File `'utils_fake_news.py'` not found.


In [25]:
path='/content/drive/MyDrive/fake-news-adversarial-benchmark/'

# Define paths liar

dataset='LIAR'
model_path = path+'Model/liar_model2'
path_csv= path+'Data/Raw/liar_valid.csv'

df=pd.read_csv(path_csv)

path_data_created='/content/drive/MyDrive/fake-news-adversarial-benchmark/data_created/liar/'


csv_path= path_data_created+'liar_1000.csv'
shap_path=path_data_created+'shapvalues_1000_liar.sav' 
path_labels=path_data_created+'labels_liar.npy'
outputs_path=path_data_created+'original_probs_liar_1000.pt'


In [11]:
# Instantiate the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained(model_path,  num_labels = 2) 

In [12]:
df_data=df.sample(1000, random_state=42)
df_data.to_csv(csv_path)

In [13]:
df_data

,ID,label,statement
1244,8905.json,1,On the Cuba embargo
331,9833.json,0,My campaign alone has created more jobs in the...
1005,1519.json,0,"Since 1981, reconciliation has been used 21 ti..."
411,7928.json,1,The state GOP ticket says their top priority i...
1204,4325.json,1,The Dodd-Frank financial-reform laws hundreds ...
...,...,...,...
942,9215.json,1,ObamaCare enrollment lie: Obama counts an enro...
287,9208.json,0,David Dewhurst currently owes more than $1 mil...
625,11612.json,1,"In 2013, the United States accepted 67 percent..."
1207,19.json,1,The president is brain-dead.


In [16]:
df_encode = encode_dataframe(df_data['statement'], df_data['label'].tolist())


The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).


##shap liar

In [17]:
# define a prediction function https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html#nlp_model
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=512, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val


In [18]:
label =[0, 1]
explainer = shap.Explainer(f, tokenizer)

In [19]:
model.to(device)
shap_values = explainer(df_data['statement'], batch_size=20)

Partition explainer: 1001it [54:29,  3.28s/it]


In [26]:
# Save shap values
pickle.dump(shap_values, open(shap_path, 'wb'))

# Save labels corresponding to SHAP values

np.save(path_labels, df_data['label'])

##liar original results

In [22]:
import itertools
def evaluate(bert_dataloader, bert_model):
    # Generate predictions
    outputs = []
    labels = []
    with torch.no_grad():
        for step, batch in enumerate(bert_dataloader):
            # Unpack batch
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # Forward pass
            output = bert_model(b_input_ids, b_input_mask)
            outputs.append(output)   

            # Keep labels
            labels.append(b_labels)     
            
    # Stack outputs
    outputs = torch.vstack([item[0].detach() for item in outputs])

    # Stack labels
    labs = [list(i.cpu().numpy()) for i in labels]
    labs = np.array(list(itertools.chain(*labs)))

    return outputs, labs

In [23]:
# Load test dataset into dataloader
batch_size = 32

dataloader = torch.utils.data.DataLoader(df_encode, batch_size=batch_size, shuffle=False)
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
# Evaluate
outputs, labs = evaluate(dataloader, model)

In [24]:
torch.save(outputs,outputs_path)